In [1]:
#load the libraries
import cv2 
from facenet_pytorch import MTCNN
import torch
import pandas as pd
import numpy as np
from PIL import Image
from tqdm.notebook import tqdm

In [2]:
v_cap = cv2.VideoCapture('C:\\Users\\lenovo\\Downloads\\The_Big_Sick-ch1.avi') #load the video by specifying the video path

In [3]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
detector = MTCNN(keep_all =True, post_process=False, device = device) #initialize the face detector on the gpu

In [4]:
# displayes the frames with bounding boxes in a window 
def show_faces(frames,predictions,i,show):
    if show == True:
        I = np.asarray(frames[i].resize([int(f * 4) for f in frame.size]))
        I = cv2.cvtColor(I, cv2.COLOR_RGB2BGR)
        if type(predictions[i]) == np.ndarray:
            for k in range(len(predictions[i])):
                x1,y1,x2,y2 = predictions[i][k]
                cv2.rectangle(I,(int(x1*4),int(y1*4)),(int(x2*4),int(y2*4)),(255,255,0),2)

        cv2.imshow('Video',I)
        cv2.putText(I, f'face detected',(20,70),cv2.FONT_HERSHEY_PLAIN, 3,(255,255,0),2)

#saves bounding boxes in a dataframe     
def save_coordinates(dataframe,predictions,j,a):
    
    dataframe.Frames.iloc[j] = 'frame{}'.format(j)
    dataframe.Coordinates.iloc[j] = predictions[a]
           
    if type(predictions[a]) == np.ndarray:
        b = []
        for y in range(len(predictions[a])):
            b.append('face{}'.format(y+1))
        dataframe.Faces.iloc[j] = b
    else:
        dataframe.Faces.iloc[j] = []
                
    dataframe.Width.iloc[j] = frames[a].width
    dataframe.Height.iloc[j] = frames[a].height

In [5]:
# initializing variables
batch_size = 32
frames = []
v_len = int(v_cap.get(cv2.CAP_PROP_FRAME_COUNT))
c=0
p=0

In [6]:
#initializing a dataframe
data = pd.DataFrame(index = np.arange(v_len),columns=['Frames','Faces','Coordinates','Width','Height'])

In [7]:
# run the detector over a batch of frames
for _ in tqdm(range(v_len)):
    # Load frame
    success, img = v_cap.read()
    if not success:
        continue
        
    # Add to batch, resizing for speed
    frame = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    frame = Image.fromarray(frame)
    frame = frame.resize([int(f * 0.25) for f in frame.size])
    frames.append(frame)
    
    # When batch is full, detect faces and reset batch list
    if len(frames) >= batch_size:
        batch_boxes, batch_probs = detector.detect(frames)
        a=0
        for j in range(c,p+1):
            save_coordinates(data,batch_boxes,j,a)
            show_faces(frames,batch_boxes,a,False)
            a+=1
            
        c = c+32
        frames = []
        
    p+=1    
    k = cv2.waitKey(1) & 0xFF
    if k == 27:
        print('User interupt')
        break
    
cv2.destroyAllWindows()

In [28]:
data.to_csv('Movie_Faces.csv',index=False) #save the dataframe